**Retrieval-Augmented Generation (RAG)** is a technique that helps improve the answers provided by large language models (like ChatGPT) by giving them additional relevant information before they generate a response. Here's a step-by-step breakdown:

-  Imagine you have a question about a specific topic (e.g., animals). You first search a special database (vector database) that stores information in a way that helps find similar concepts.

- Retrieve Relevant Information: The database returns the most relevant pieces of information related to your query (e.g., facts about elephants and other animals).

- Build a Prompt: You then create a prompt (a set of instructions) that includes the retrieved information. This is like gathering notes before writing an essay.

- Generate an Answer: Finally, you feed this prompt into the language model (like ChatGPT), which uses the provided information to generate a more accurate and informed answer.


In simple terms, RAG is like doing a quick research before answering a question, ensuring that the response is based on the most relevant and up-to-date information available. This helps the language model provide better and more accurate answers, even if it wasn't originally trained on that specific information.

In [50]:
import requests
import os
import json
import weaviate
from weaviate.embedded import EmbeddedOptions

In [51]:
# download the data
# Downlaod the data https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json
response = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(response.text)

# Parse the json and preview the data using json_prin function
def json_print(data):
    print(json.dumps(data, indent=2))

print(type(data))
print(len(data))
print(json.dumps(data[0], indent=2))

<class 'list'>
10
{
  "Category": "SCIENCE",
  "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "Answer": "Liver"
}


In [52]:
# Load the data into Weaviate
cohere_api_key = os.getenv("COHERE_APIKEY")

waeivate_api_key = os.getenv("WEAVIATE_API_KEY")

openai_api_key = os.getenv("OPENAI_API")

auth_config = weaviate.AuthApiKey(api_key=waeivate_api_key)

# Connect to the locallay launched instance of Weaviate
client = weaviate.Client(
        url = "http://localhost:8080",
        additional_headers={
        "X-Cohere-Api-Key": cohere_api_key
        #"X-Google-Studio-Api-Key": ai_studio_api_key
        #"X-OpenAI-Api-Key": openai_api_key
    }

    )

print(f"Client is ready: {client.is_ready()}")

"""client = weaviate.Client(
    url="https://e2pxfwhqioinxijlmnqxw.c0.europe-west3.gcp.weaviate.cloud",
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": cohere_api_key
        #"X-Google-Studio-Api-Key": ai_studio_api_key
        #"X-OpenAI-Api-Key": openai_api_key
    }
)"""

#print("client created")

d:\Vector Databases\vector-databases-certification\env\Lib\site-packages\weaviate\__init__.py:143: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])
d:\Vector Databases\vector-databases-certification\env\Lib\site-packages\weaviate\warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


Client is ready: True


'client = weaviate.Client(\n    url="https://e2pxfwhqioinxijlmnqxw.c0.europe-west3.gcp.weaviate.cloud",\n    auth_client_secret=auth_config,\n    additional_headers={\n        "X-Cohere-Api-Key": cohere_api_key\n        #"X-Google-Studio-Api-Key": ai_studio_api_key\n        #"X-OpenAI-Api-Key": openai_api_key\n    }\n)'

In [53]:
# Check and delete already existed class
if client.schema.exists("Question"):
    client.schema.delete_class("Question")
print("Class Deleted Successfully")

Class Deleted Successfully


In [55]:
# Create class object
class_obj = {
    "class": "Question",
    "vectorizer": "text2vec-cohere",
}

client.schema.create_class(class_obj)
print("Class created")

Class created


In [56]:
with client.batch.configure() as batch:
    for i, d in enumerate(data):  # Batch import data
        
        print(f"importing question: {i+1}")
        
        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }
        
        batch.add_data_object(
            data_object=properties,
            class_name="Question"
        )

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10
{'error': [{'message': 'Cohere API Key: no api key found neither in request header: X-Cohere-Api-Key nor in environment variable under COHERE_APIKEY'}]}
{'error': [{'message': 'Cohere API Key: no api key found neither in request header: X-Cohere-Api-Key nor in environment variable under COHERE_APIKEY'}]}
{'error': [{'message': 'Cohere API Key: no api key found neither in request header: X-Cohere-Api-Key nor in environment variable under COHERE_APIKEY'}]}
{'error': [{'message': 'Cohere API Key: no api key found neither in request header: X-Cohere-Api-Key nor in environment variable under COHERE_APIKEY'}]}
{'error': [{'message': 'Cohere API Key: no api key found neither in request header: X-Cohere-Api-Key nor in environment variable under COHERE_APIKEY'}]}
{'error': [{'m

In [ ]:
json_print(client.query.aggregate('Question').with_meta_count().do())

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 0
          }
        }
      ]
    }
  }
}


In [ ]:
# write a query for vector search related to the concept animals
repsonse = (
    client.query.get('Question', 'answer')
    .with_near_text({"concepts": "animals"})
    .with_limit(5)
    .do()
)

print(json.dumps(repsonse, indent=2))

{
  "data": {
    "Get": {
      "Question": null
    }
  },
  "errors": [
    {
      "locations": [
        {
          "column": 6,
          "line": 1
        }
      ],
      "message": "explorer: get class: vectorize params: vectorize params: vectorize params: vectorize keywords: remote client vectorize: API Key: no api key found neither in request header: X-Openai-Api-Key nor in environment variable under OPENAI_APIKEY",
      "path": [
        "Get",
        "Question"
      ]
    }
  ]
}


In [ ]:
# Pass each of these objects to a LLM individually to use when answering a prompt

# write a prompt that uses the animal object and passes it to the LLM
prompt = "Tell me a story about this animal {answer}!"

In [ ]:
# Write a query to perform RAG
response = (client.query
            .get("Question", "answer")
            .with_near_text({"concepts": "animals"})
            .with_generate(single_prompt="Tell me a story about this animal {answer}!")
            .with_limit(5)
            .do()
            )

json_print(response)

{
  "errors": [
    {
      "locations": [
        {
          "column": 80,
          "line": 1
        }
      ],
      "message": "Cannot query field \"generate\" on type \"QuestionAdditional\".",
      "path": null
    }
  ]
}


In [ ]:
# Extract All Categories
response = (client.query
            .get("Question", 'category')
            .with_near_text({'concepts':'animals'})
            .with_limit(10)
            .do()
           )

json_print(response)

{
  "data": {
    "Get": {
      "Question": null
    }
  },
  "errors": [
    {
      "locations": [
        {
          "column": 6,
          "line": 1
        }
      ],
      "message": "explorer: get class: vectorize params: vectorize params: vectorize params: vectorize keywords: remote client vectorize: API Key: no api key found neither in request header: X-Openai-Api-Key nor in environment variable under OPENAI_APIKEY",
      "path": [
        "Get",
        "Question"
      ]
    }
  ]
}
